In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import het_breuschpagan
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error


np.random.seed(0)

In [ ]:
df = pd.read_csv("./datasets/wdbc.data", header=None).drop(columns=[0])
df_benign = df[df[1] == 'B'].drop(columns=[1])

col_names = [
    "radius",
    "texture",
    "perimeter",
    "area",
    "smoothness",
    "compactness",
    "concavity",
    "concave",
    "symmetry",
    "fractal"
]
columns = list(range(2, 12))
columns_dict = dict(zip(columns, col_names))

df_benign = df_benign[columns].rename(columns=columns_dict)

In [ ]:
df_benign

In [ ]:
perfect_linear_corr_set = ['radius', 'perimeter']
biased_linear_corr_set = ['compactness', 'concavity']

fig, ax = plt.subplots(1, 2, figsize=(15, 6))

df_linear = df_benign[perfect_linear_corr_set].sort_values(by=perfect_linear_corr_set[0]).reset_index(drop=True)
df_biased_linear = df_benign[biased_linear_corr_set].sort_values(by=biased_linear_corr_set[0]).reset_index(drop=True)

def display_linear(ax, ax_i, df_linear):
    ax[ax_i].scatter(df_linear.values[:, 0], df_linear.values[:, 1])
    ax[ax_i].set_title("Almost perfect linear correlation")
    ax[ax_i].set_xlabel(perfect_linear_corr_set[0])
    ax[ax_i].set_ylabel(perfect_linear_corr_set[1])
    x_min, x_max = df_linear.values[:, 0].min(), df_linear.values[:, 0].max()
    y_min, y_max = df_linear.values[:, 1].min(), df_linear.values[:, 1].max()
    ax[ax_i].set_xticks(np.arange(np.floor(x_min), np.ceil(x_max) + 1, 1))
    ax[ax_i].set_yticks(np.arange(np.floor(y_min), np.ceil(y_max) + 1, 2))

def display_biased_linear(ax, ax_i, df_biased_linear):
    ax[ax_i].scatter(df_biased_linear.values[:, 0], df_biased_linear.values[:, 1])
    ax[ax_i].set_title("Biased linear correlation")
    ax[ax_i].set_xlabel(biased_linear_corr_set[0])
    ax[ax_i].set_ylabel(biased_linear_corr_set[1])
    x_min, x_max = df_biased_linear.values[:, 0].min(), df_biased_linear.values[:, 0].max()
    y_min, y_max = df_biased_linear.values[:, 1].min(), df_biased_linear.values[:, 1].max()
    ax[ax_i].set_xticks(np.linspace(x_min, x_max, num=10))
    ax[ax_i].set_yticks(np.linspace(y_min, y_max, num=10))

display_linear(ax, 0, df_linear)
display_biased_linear(ax, 1, df_biased_linear)


In [ ]:
def build_and_analyze_model(df, x_col, y_col, biased=False, outlier_threshold=3):
    X = sm.add_constant(df[x_col])
    y = df[y_col]

    model = sm.OLS(y, X)
    model_results = model.fit()

    print(f"\n\n{"="*10} Correlation: {"="*10}")
    correlation_value = df[x_col].corr(df[y_col], method='pearson')  # ((x - x.mean()) * (y - y.mean())).mean() / (x.std() * y.std())
    print(f'Pearson correlation coefficient between `{x_col}` and `{y_col}`: {correlation_value}')
    print("="*60)
    
    print(f"\n\n{"="*10} Model summary description: {"="*10}")
    print(model_results.summary())
    print("="*60)
    
    residuals = model_results.resid
    fitted_values = model_results.fittedvalues


    print(f"\n\n{"="*10} Residuals squared error: {"="*10}")
    rse = np.mean(residuals ** 2)
    print(f'RSE: {rse}')
    print("="*60)

    print(f"\n\n{"="*10} Residuals statistics description: {"="*10}")
    display(residuals.describe())
    print("="*60)

    fig, ax = plt.subplots(3, 3, figsize=(18, 18))
    ax = ax.flatten()

    if biased:
        display_biased_linear(ax, 0, df)
    else:
        display_linear(ax, 0, df)
    xs = np.linspace(df[x_col].min(), df[x_col].max(), 2)
    ys = model_results.params[x_col] * xs + model_results.params['const']
    ax[0].plot(xs, ys, color='red')
    ax[0].set_title("Regression line")

    sns.histplot(residuals, kde=True, ax=ax[1])
    ax[1].set_title('Histogram residuals')
    
    scipy.stats.probplot(residuals, dist="norm", plot=ax[2])
    ax[2].set_title('Q-Q plot residuals')

    ax[3].scatter(fitted_values, residuals, edgecolors='k', facecolors='none')
    ax[3].axhline(0, color='gray', linestyle='dashed', linewidth=2)
    ax[3].set_title("Residuals vs fitted")
    ax[3].set_xlabel("Fitted values")
    ax[3].set_ylabel("Residuals")
    z = sm.nonparametric.lowess(residuals, fitted_values)
    ax[3].plot(z[:, 0], z[:, 1], color='red', lw=2)


    sqrt_standardized_residuals = np.sqrt(np.abs(residuals / np.std(residuals)))
    ax[4].scatter(fitted_values, sqrt_standardized_residuals, edgecolors='k', facecolors='none')
    z = sm.nonparametric.lowess(sqrt_standardized_residuals, fitted_values)
    ax[4].plot(z[:, 0], z[:, 1], color='red', lw=2)
    ax[4].set_title("Scale-location")
    ax[4].set_xlabel("Fitted values")
    ax[4].set_ylabel("√|Standardized residuals|")


    cooks_d = model_results.get_influence().cooks_distance[0]
    outlier_cooks_threshold = (4 / (cooks_d.size - 1))
    outliers_cooks_d = np.where(cooks_d > outlier_cooks_threshold)[0]
    ax[5].plot(cooks_d, 'bo', linestyle='None')
    ax[5].axhline(outlier_cooks_threshold, color='red', linestyle='dashed', label=f"Threshold: {outlier_cooks_threshold}")
    ax[5].set_title("Cook's Distance")
    ax[5].set_xlabel("Observation Index")
    ax[5].set_ylabel("Cook's Distance")

    standardized_residuals = np.abs(residuals / np.std(residuals))
    ax[6].boxplot(standardized_residuals)
    ax[6].set_title('Boxplot of Standardized Residuals')
    ax[6].set_ylabel('Standardized Residuals')

    # Wartości odstające i wpływowe
    sm.graphics.influence_plot(model_results, criterion="cooks", ax=ax[7])

    
    
    
    print(f"\n\n{"="*10} Residual normality check: {"="*10}")
    print("Shapiro-Wilk test p-value:", scipy.stats.shapiro(residuals)[1])
    print("="*60)

    print(f"\n\n{"="*10} Residual skewness check: {"="*10}")
    print("Skewness:", pd.Series(residuals).skew())
    print("="*60)

    print(f"\n\n{"="*10} Residual kurtosis check: {"="*10}")
    print("Kurtosis:", pd.Series(residuals).kurtosis())
    print("="*60)

    
    # Homoskedastyczność (test Levene'a)
    print(f"\n\n{"="*10} Homoscedasticity check: {"="*10}")
    median = np.median(fitted_values)
    group1 = residuals[fitted_values <= median]
    group2 = residuals[fitted_values > median]
    levene_test = scipy.stats.levene(group1, group2)
    print("Levene's test p-value:", levene_test.pvalue)
    print("="*60)

    # Test Box-Pierce
    print(f"\n\n{"="*10} Autocorrelation check (for lags=1,2,3): {"="*10}")
    boxpierce_results = sm.stats.diagnostic.acorr_ljungbox(residuals, lags=3, boxpierce=True)
    display(boxpierce_results)
    print("Box-Pierce test statistic:", boxpierce_results['bp_stat'].values)
    print("Box-Pierce test p-value:", boxpierce_results['bp_pvalue'].values)
    print("="*60)


    print(f"\n\n{"="*10} Outliers list: {"="*10}")
    print(f"\nIndices of outliers based on cooks distance (beyond threshold {outlier_cooks_threshold}): {outliers_cooks_d}")
    print("="*60)

    print(f"\n\n{"="*10} Correlation coefficient statistical significance: {"="*10}")
    r2 = model_results.rsquared
    stat = r2 / np.sqrt((1 - r2) / (df.shape[0]-2))
    crit = scipy.stats.t.ppf(1 - 0.05/2, df.shape[0]-2)
    print(f"Statystyka t: {stat} \nWartość krytyczna (górna): {crit}")
    print("="*60)

    plt.legend()
    plt.show()

    return outliers_cooks_d

<center>
<h4>
Badanie liniowej zależnosci miedzy `radius` a `perimeter`
</h4>
</center>

Wiemy, że zależność miedzy promieniem a obwodem jest liniowa z wzoru 2PIr. Oczekujemy prawie perfekcyjnej liniowej zależności, która będzie 
zaburzona najprawdopodobnije błędami pomiaru. Promień może być jedynie dodatni oraz jego wielkość również nie moze przekraczać, powiedzmy 40cm.

In [ ]:
df_linear = df_linear[(df_linear.radius > 0) & (df_linear.radius < 40)]

In [ ]:
outliers = build_and_analyze_model(df_linear, 'radius', 'perimeter')

- Korelacja wynosi 0.99 co świadczy o bardzo wysokiej zależności między tymi dwoma zmiennymi. Dopasowany model ma postać funkcyjną \
y = 6.61 * x + -2.21 \
i został dopasowany na 357 obserwacjach. 
- Wartości reziduów mają rozkład normalny, prawoskośny. O normalności świadczy test saphiro-wilka,
którego p-value < 0.05 (qq-plot wykazuje minimalne odchylenie od normalności, ponieważ nie wsyzskite wartości leżą ideealnie na linii.)
- Dodatnie Skewness (1.32) wskazuje na prawostronną skośność, co widoczne jest też na histogramie - sporo predykcji dla których otrzymujemy zbyt duze wartości. Kurtoza ma wartość 4 co oznaczałoby
grube ogony w rozkladzie reziduów. \
- Rozklad reziduów ma mean~0 oraz std~1
- Dla `const` i `radius` mamy hipotezy zerowe H_0 mówiące o tym, że współczynnik jest równy zero. Dla obu wartości t nie wpada w 95%-przedział
ufności, zatem odrzucamy takie hipotezy na rzecz hipotezy H_1 mówiącej o tym, że współczynnik nie jest równy 0. OBA WSPÓŁCZYNNIKI SĄ ISTOTNE STATYSTYCZNIE.
- RSE wynosi 0.8969853117760591 co oznacza, że średni błąd kwadratowy jest równy 0.8969853117760591. Jest to akceptowalna wartość, ponieważ opeujemy na dużym zakresie wartości (ok 44-100)
- **Model wyjaśnia 99% wariancji zmiennej y przez zmienną x** ponieważ R^2 = 0.99
- p-value dla F-statistic jest równe 0 (< 0.05), co interpretujemy jako niemożliwość wyjaśnienia wartości y za pomocą wzoru y = b. Musimy użyć y = ax + b

- Z wykresu Residuals vs Fitted wynika, że nie zachodzi trend w rozkladzie wartosci reziduów. Wykres uklada sie równomiernie
wokół prostej y=0
- Test Levene'a wykazuje p-value < 0.05, co mówiłoby o homoskedastyczności reziduów. Oznacza to, że mamy stałe wartości reszt wzdłuż
wartości przewidywanych przez model. Rozrzut wartości reszt wokół linii regresji jest stały na wszystkich wartościach x (dla róznych przedziałów).
- Wykres Scale-location informuje nas, ze pierwiastek z ustandaryzowanych reszt jest praktycznie zawsze stały dla wszystkich przedziałów (wartości)
zmiennej niezależnej X (tj. dla wszystkich dopasowanych wartości y)

- Nie występuje autokorelacja reszt składnika losowego (reszty nie są ze sobą skorelowane), ponieważ nie odrzucamy hipotezy H_0
mówiącej o niewystępowaniu autokorelacji. Na poziomach lag=1,2,3 otrzymujemy p-value z testu Boxa-Pierca: [0.08561466 0.17790812 0.15246615]
które są >0.05. Zatem potwierdzamy H_0.

- Do nastepnych eksperymentów odrzucamy obserwacje znaczące (wpływowe), tj. dla których odległość cook'a była większa od 0.011235955056179775 (na podstawie heurystyki 4/(n-1))

- Statystyka t jest większa od wartości krytycznej dla n-2 stopni swobody, gdzie n jest liczbą obserwacji. Odrzucamy hipotezę zerową, 
że korelacja jest równa 0

<center>
<h5>
Teraz wykonamy eksperymenty dla tych samych zmiennych, ale z odfiltrowanymi obserwacjami wpływowymi na podstawie odległości cooka
</h5>
</center>

In [ ]:
df_linear_filtered = df_linear.drop(outliers)
build_and_analyze_model(df_linear_filtered, 'radius', 'perimeter')

Jak widać :
- QQ plot przyjął teraz bardziej liniowy kształt, 
- histogram reszt nie jest juz prawoskośny (nie ma obserwacji odstających w prawym ogonie),
- influence plot nie posiada juz tak zróżnicowanych wartości na osi X
- box plot nie ma tak zróżnicowanych wartości, chociaz jeszcze zostały 3 outliery, 
- skala w Residuals vs fitted nie jest już tak zróżnicowana (największe wartości to 2, a nie 6 jak poprzednio)

In [ ]:
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import r2_score

def cross_validate_ols(df, x_cols, y_col, n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    r2_scores = []
    rse_scores = []

    for train_index, test_index in kf.split(df):
        X_train, X_test = df[x_cols].iloc[train_index], df[x_cols].iloc[test_index]
        y_train, y_test = df[y_col].iloc[train_index], df[y_col].iloc[test_index]

        X_train = sm.add_constant(X_train)
        model = sm.OLS(y_train, X_train)
        model_results = model.fit()

        X_test = sm.add_constant(X_test)
        y_pred =  (model_results.params[x_cols[0]] * X_test[x_cols[0]] +  model_results.params['const']).values.reshape(-1)
        y_test = y_test.values.reshape(-1)

        r2 = r2_score(y_test, y_pred)
        r2_scores.append(r2)

        residuals = y_test - y_pred

        rse = np.mean(residuals ** 2)
        rse_scores.append(rse)

    mean_r2 = np.mean(r2_scores)
    mean_rse = np.mean(rse_scores)

    return mean_r2, mean_rse

def plot_train_test_reg(df, x_cols, y_col):
    X = df[x_cols]
    y = df[y_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    X_train = sm.add_constant(X_train)
    model = sm.OLS(y_train, X_train)
    model_results = model.fit()
    xs = np.linspace(X_train[x_cols[0]].min(), X_train[x_cols[0]].max(), 2)
    ys = model_results.params[x_cols[0]] * xs + model_results.params['const']
    
    plt.scatter(X_train[x_cols[0]], y_train, color='blue', label='Train')
    plt.scatter(X_test, y_test, color='red', label='Test')
    plt.plot(xs, ys, color='purple')
    plt.xlabel(x_cols)
    plt.ylabel(y_col)
    plt.legend()
    plt.show()




In [ ]:
cv_r2, cv_rse = cross_validate_ols(df_linear_filtered, ['radius'], ['perimeter'])
print(f"R2 for 10-fold cross validation: {cv_r2} \nRSE for 10-fold cross validation: {cv_rse}")

In [ ]:
plot_train_test_reg(df_linear_filtered, ['radius'], ['perimeter'])

<center>
<h4>
Badanie liniowej zależnosci miedzy `compactness` a `concavity`
</h4>
</center>

W tym zbiorze widac pewną korelacje obu zmiennych i chcemy sprawdzić, czy można na nim sensownie dopasować model regresji liniowej.
Oczekujemy umiarkowanej liniowej zależności ze sporymi reziduami. Są to wartosci, które powinny przyjmowac wartości dodatnie więc 
przyjmujemy `comcpactness` > 0.

In [ ]:
df_biased_linear = df_biased_linear[(df_biased_linear.compactness > 0)]


In [ ]:
outliers = build_and_analyze_model(df_biased_linear, 'compactness', 'concavity', biased=True)

<center>
<h5>
Teraz wykonamy eksperymenty dla tych samych zmiennych, ale z odfiltrowanymi obserwacjami wpływowymi na podstawie odległości cooka
</h5>
</center>

In [ ]:
df_biased_linear_filtered = df_biased_linear.drop(outliers)
build_and_analyze_model(df_biased_linear_filtered, 'compactness', 'concavity', biased=True)

In [ ]:
cv_r2, cv_rse = cross_validate_ols(df_biased_linear_filtered, ['compactness'], ['concavity'])
print(f"R2 for 10-fold cross validation: {cv_r2} \nRSE for 10-fold cross validation: {cv_rse}")

In [ ]:
plot_train_test_reg(df_biased_linear_filtered, ['compactness'], ['concavity'])